In [4]:
import numpy as np
import pandas as pd

In [5]:
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [7]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
label_encoding_gender=LabelEncoder()
onehot_encoder=OneHotEncoder()
df['Gender']=label_encoding_gender.fit_transform(df['Gender'])
encoded=onehot_encoder.fit_transform(df[['Geography']]).toarray()

In [9]:
df.drop('Geography',axis=1,inplace=True)

In [10]:
df1=pd.DataFrame(encoded,columns=onehot_encoder.get_feature_names_out())

In [11]:
df=pd.concat([df,df1],axis=1)

In [12]:
x=df.drop('Exited',axis=1)
y=df['Exited']

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=44)

##Scale the features
from sklearn.preprocessing import StandardScaler
scale=StandardScaler()
x_train=scale.fit_transform(x_train)
x_test=scale.transform(x_test)

In [14]:
import pickle
with open('scaler.pkl','wb') as file:
    pickle.dump(scale,file)
with open('labelencoder.pkl','wb') as file:
    pickle.dump(label_encoding_gender,file)
with open('onehotencoder.pkl','wb') as file:
    pickle.dump(onehot_encoder,file)

## ANN Implementation

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [18]:
(x_train.shape[1],)

(12,)

In [19]:
## Building your model
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),## hiden layer 1
    Dense(32,activation='relu'),## hiden layer 2
    Dense(1,activation='sigmoid') ## Output layer
])

c:\Users\mohdz\OneDrive\Desktop\python\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
loss=tf.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x000002366ED37BA0>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [26]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
##OR
# model.compile(optimizer=opt,loss=loss,metric=['accuray'])

In [ ]:
## Set Up the TensorBoard
log_dir= 'logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callbacks=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [41]:
## Set Up the Early stopping
early_stop=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [42]:
## Traning the model
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100,
          callbacks=[tensorboard_callbacks,early_stop])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8709 - loss: 0.3142 - val_accuracy: 0.8630 - val_loss: 0.3320
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8695 - loss: 0.3142 - val_accuracy: 0.8645 - val_loss: 0.3321
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8719 - loss: 0.3109 - val_accuracy: 0.8580 - val_loss: 0.3306
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8704 - loss: 0.3114 - val_accuracy: 0.8620 - val_loss: 0.3269
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8709 - loss: 0.3094 - val_accuracy: 0.8550 - val_loss: 0.3328
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8740 - loss: 0.3080 - val_accuracy: 0.8635 - val_loss: 0.3340
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8729 - loss: 0.3088 - val_accuracy: 0.8570 - val_loss: 0.3280
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8736 - loss: 0.3058 - val_accu

In [43]:
model.save('model.h5')

In [44]:
## load Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [45]:
%tensorboard --logdir logs/fit/20251001-221521

Reusing TensorBoard on port 6011 (pid 14296), started 0:12:55 ago. (Use '!kill 14296' to kill it.)